In [12]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("./log/1mo.pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml'])

dict

In [13]:
game_info = data["ml"]["scene_info"]
game_command = data["ml"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'snake_head': (40, 40), 'snake_body': [(40, 30), (40, 20), (40, 10)], 'food': (100, 250)}, {'frame': 1, 'status': 'GAME_ALIVE', 'snake_head': (40, 50), 'snake_body': [(40, 40), (40, 30), (40, 20)], 'food': (100, 250)}, {'frame': 2, 'status': 'GAME_ALIVE', 'snake_head': (50, 50), 'snake_body': [(40, 50), (40, 40), (40, 30)], 'food': (100, 250)}, {'frame': 3, 'status': 'GAME_ALIVE', 'snake_head': (60, 50), 'snake_body': [(50, 50), (40, 50), (40, 40)], 'food': (100, 250)}, {'frame': 4, 'status': 'GAME_ALIVE', 'snake_head': (70, 50), 'snake_body': [(60, 50), (50, 50), (40, 50)], 'food': (100, 250)}, {'frame': 5, 'status': 'GAME_ALIVE', 'snake_head': (80, 50), 'snake_body': [(70, 50), (60, 50), (50, 50)], 'food': (100, 250)}, {'frame': 6, 'status': 'GAME_ALIVE', 'snake_head': (90, 50), 'snake_body': [(80, 50), (70, 50), (60, 50)], 'food': (100, 250)}, {'frame': 7, 'status': 'GAME_ALIVE', 'snake_head': (100, 50), 'snake_body': [(90, 50), (80, 50), (70, 5

In [14]:
for i in range(2,169): 
    path = "./log/" + str(i) + "mo.pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml']['scene_info']
    game_command = game_command + data['ml']['command']
    file.close()  

print(len(game_info))
print(len(game_command))

243488
243488


In [15]:
g = game_info[1]

feature = np.array([g["snake_head"][0], g["snake_head"][1], 0,0,0, g["food"][0],g["food"][1],0,0])
print(feature)
print(game_command[1])
if game_command[1] == "UP": game_command[1] = 0
elif game_command[1] == "DOWN": game_command[1] = 1
elif game_command[1] == "LEFT": game_command[1] = 2
elif game_command[1] == "RIGHT": game_command[1] = 3
else: game_command[1] = 4 

[ 40  50   0   0   0 100 250   0   0]
RIGHT


In [16]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    snakehead_x=g["snake_head"][0]
    snakehead_y=g["snake_head"][1]
    x_dir=g['snake_head'][0] - g_last['snake_head'][0]
    y_dir=g['snake_head'][1] - g_last['snake_head'][1]
    food_x=g["food"][0]
    food_y=g["food"][1]
    if x_dir>0 and y_dir == 0:
        direction = 4 
        wall_disx = 300 - g['snake_head'][0]
    elif x_dir<0 and y_dir == 0:
        direction = 3
        wall_disx = g['snake_head'][0]
    elif y_dir>0 and x_dir == 0:
        direction = 2
        wall_disy = 300 - g['snake_head'][1]
    elif y_dir<0 and x_dir == 0:
        direction = 1
        wall_disy = g['snake_head'][1]
        
    if y_dir == 0:
        wall_disy = max(300 - g['snake_head'][1],g['snake_head'][1])
    elif x_dir == 0:
        wall_disx = max(300 - g['snake_head'][0],g['snake_head'][0])
    body = 0
    wall = 0
    #wall            
    #四個角角
    snake_head = g["snake_head"]
    if ((snake_head[0] > 285 and snake_head[1] > 285)or(snake_head[0] > 285 and snake_head[1] < 5) or (snake_head[0] < 5 and snake_head[1] > 285)or(snake_head[0] < 5 and snake_head[1] > 285)):
        #x方向移動
        if direction >= 3:
            if snake_head[0] > 285:
                if snake_head[1] > 285: #右下
                    wall = 1
                if snake_head[1] < 5:#右上
                    wall = 2
            if snake_head[0] < 5:
                if snake_head[1] > 285:#左下
                    wall = 1
                if snake_head[1] < 5:#左上
                    wall = 2
        #y方向移動
        if direction <= 2:
            if snake_head[1] > 285:
                if snake_head[0] > 285: #右下
                    wall = 3
                if snake_head[0] < 5:#左下
                    wall = 4
            if snake_head[1] < 5:
                if snake_head[0] > 285:#右上
                    wall = 3
                if snake_head[0] < 5:#左下
                    wall = 4          
    else:
                #x方向移動
        if direction >= 3:
            if snake_head[0] >= 285:
                if snake_head[1] > 145: #右下
                    wall = 1
                else:#右上
                    wall = 2
            if snake_head[0] < 5:
                if snake_head[1] > 145:#左下
                    wall = 1
                else:#左上
                    wall = 2
                #y方向移動
        if direction <= 2:
            if snake_head[1] >= 285:
                if snake_head[0] > 145: #右下
                    wall = 3
                else:#左下
                    wall = 4
            if snake_head[1] < 5:
                if snake_head[0] > 145:#右上
                    wall = 3
                else:#左上
                    wall = 4


    #是否離body直線距離<=10?body left==3,right==4,up==1,down==2
    for m in range(0,len(g['snake_body'])):
        if direction==3:#left
            if(g['snake_head'][0]==g['snake_body'][m][0]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]==g['snake_body'][m][1]+10):
                        body = 2
                    elif(g['snake_head'][1]==g['snake_body'][m][1]-10):
                        body = 1
            if (g['snake_head'][0]==g['snake_body'][m][0]+10 and g['snake_head'][1]==g['snake_body'][m][1]): 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]+10==g['snake_body'][m+1][1]):
                        body = 2
                    elif(g['snake_head'][1]-10==g['snake_body'][m+1][1]):
                        body = 1
                        
                    
        elif direction==4:#right 
            if(g['snake_head'][0]==g['snake_body'][m][0]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]==g['snake_body'][m][1]+10):
                        body = 2
                    elif(g['snake_head'][1]==g['snake_body'][m][1]-10):
                        body = 1
            if (g['snake_head'][0]==g['snake_body'][m][0]-10 and g['snake_head'][1]==g['snake_body'][m][1]): # 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]+10==g['snake_body'][m+1][1]):
                        body = 2
                    elif(g['snake_head'][1]-10==g['snake_body'][m+1][1]):
                        body = 1
                      
                            
        elif direction==1:#up
            if(g['snake_head'][1]==g['snake_body'][m][1]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]==g['snake_body'][m][0]+10):
                        body = 4
                    elif(g['snake_head'][0]==g['snake_body'][m][0]-10):
                        body = 3
            if (g['snake_head'][1]==g['snake_body'][m][1]+10 and g['snake_head'][0]==g['snake_body'][m][0]): #
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]+10==g['snake_body'][m+1][0]):
                        body = 4
                    elif(g['snake_head'][0]-10==g['snake_body'][m+1][0]):
                        body = 3
                        
        elif direction==2:#down
            if(g['snake_head'][1]==g['snake_body'][m][1]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]==g['snake_body'][m][0]+10):
                        body = 4
                    elif(g['snake_head'][0]==g['snake_body'][m][0]-10):
                        body = 3
            if (g['snake_head'][1]==g['snake_body'][m][1]-10 and g['snake_head'][0]==g['snake_body'][m][0]): # 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]+10==g['snake_body'][m+1][0]):
                        body = 4
                    elif(g['snake_head'][0]-10==g['snake_body'][m+1][0]):
                        body = 3
                        

    
    feature = np.vstack((feature, [g["snake_head"][0], g["snake_head"][1],direction,wall_disx,wall_disy,food_x,food_y,body,wall]))
    if game_command[i] == "UP": game_command[i] = 0
    elif game_command[i] == "DOWN": game_command[i] = 1
    elif game_command[i] == "LEFT": game_command[i] = 2
    elif game_command[i] == "RIGHT": game_command[i] = 3
    else: game_command[i] = 4 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 40  50   0 ... 250   0   0]
 [ 50  50   4 ... 250   0   0]
 [ 60  50   4 ... 250   0   0]
 ...
 [ 70 130   4 ... 120   0   0]
 [ 80 130   4 ... 120   0   0]
 [ 90 130   4 ... 120   0   0]]
(243486, 9)
[3 3 3 ... 3 3 3]
(243486,)


# KNN


In [35]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=0)
#參數區間
param_grid = {'n_neighbors':[0, 1, 2, 3, 4]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(n_neighbors = 5), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model_knn.pickle', 'wb')
pickle.dump(grid, file)
file.close()


Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:    4.0s remaining:   12.1s
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    4.1s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:    4.1s remaining:    4.1s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    6.2s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:    6.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    6.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    6.4s finished


In [36]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 3}
[[ 8127    98   417   465     0]
 [  145  8355   387   452     0]
 [  511   393 11803   151     0]
 [  459   439   165 11697     0]
 [    7    11    12    17    10]]
              precision    recall  f1-score   support

           0       0.88      0.89      0.89      9107
           1       0.90      0.89      0.90      9339
           2       0.92      0.92      0.92     12858
           3       0.92      0.92      0.92     12760
           4       1.00      0.18      0.30        57

    accuracy                           0.91     44121
   macro avg       0.92      0.76      0.78     44121
weighted avg       0.91      0.91      0.91     44121



# MLPClassifier

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
MLPC = MLPClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

MLPC.predict_proba(x_test)
MLPC.predict(x_test)
MLPC.score(x_test, y_test)

file = open("model_MLPC.pickle_168", "wb")
pickle.dump(MLPC, file)
file.close()

In [ ]:
print("Accuracy score (validation): {0:.3f}".format(MLPC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, MLPC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, MLPC.predict(x_test)))

Accuracy score (validation): 0.993
Confusion Matrix for Raandom Forests:
[[15327    11     1     1     0]
 [  221 15430     1     0     2]
 [    0     0 20878     3     7]
 [    2     1   135 20906    28]
 [   16    15    20    16    25]]

Classification Report for Random Forests


# HistGradientBoostingClassifier (會delay不要用)

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
HGBC = HistGradientBoostingClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

HGBC.predict_proba(x_test)
HGBC.predict(x_test)
HGBC.score(x_test, y_test)

file = open("model_HGBC.pickle", "wb")
pickle.dump(HGBC, file)
file.close()

In [12]:
print("Accuracy score (validation): {0:.3f}".format(HGBC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, HGBC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, HGBC.predict(x_test)))

Accuracy score (validation): 0.989
Confusion Matrix for Raandom Forests:
[[ 8965    19     8    33    28]
 [   24  9324     2     7    11]
 [   56    73 12714     7    15]
 [   54    85     3 12610    24]
 [    2     7     9    14    27]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9053
           1       0.98      1.00      0.99      9368
           2       1.00      0.99      0.99     12865
           3       1.00      0.99      0.99     12776
           4       0.26      0.46      0.33        59

    accuracy                           0.99     44121
   macro avg       0.84      0.88      0.86     44121
weighted avg       0.99      0.99      0.99     44121



# DecisionTree

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
DT = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)

DT.predict_proba(x_test)
DT.predict(x_test)
DT.score(x_test, y_test)

file = open("model_DT.pickle", "wb")
pickle.dump(DT, file)
file.close()

In [9]:
print("Accuracy score (validation): {0:.3f}".format(DT.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, DT.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, DT.predict(x_test)))

Accuracy score (validation): 0.992
Confusion Matrix for Raandom Forests:
[[13861    93     8    16    10]
 [   88 14521    10    10    12]
 [   55    53 19180    19     9]
 [   44    61    27 19303     7]
 [    7    13     8    11    41]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     13988
           1       0.99      0.99      0.99     14641
           2       1.00      0.99      1.00     19316
           3       1.00      0.99      0.99     19442
           4       0.52      0.51      0.52        80

    accuracy                           0.99     67467
   macro avg       0.90      0.90      0.90     67467
weighted avg       0.99      0.99      0.99     67467



# Extra tree

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import ExtraTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
cls = ExtraTreeClassifier(random_state=0)
ET = BaggingClassifier(cls, random_state=0).fit(x_train, y_train)

ET.predict_proba(x_test)
ET.predict(x_test)
ET.score(x_test, y_test)

file = open("model_ET.pickle", "wb")
pickle.dump(ET, file)
file.close()

In [11]:
print("Accuracy score (validation): {0:.3f}".format(ET.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, ET.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, ET.predict(x_test)))

Accuracy score (validation): 0.987
Confusion Matrix for Raandom Forests:
[[13857    46    35    48     2]
 [   75 14474    42    48     2]
 [  138   124 19041    13     0]
 [   91   119    24 19207     1]
 [    9    20    16    18    17]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     13988
           1       0.98      0.99      0.98     14641
           2       0.99      0.99      0.99     19316
           3       0.99      0.99      0.99     19442
           4       0.77      0.21      0.33        80

    accuracy                           0.99     67467
   macro avg       0.94      0.83      0.86     67467
weighted avg       0.99      0.99      0.99     67467



# AdaBoostClassifier

In [27]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)

ABC = AdaBoostClassifier(n_estimators=100, random_state=0).fit(x_train, y_train)

ABC.predict_proba(x_test)
ABC.predict(x_test)
ABC.score(x_test, y_test)

file = open("model_ABC.pickle", "wb")
pickle.dump(ABC, file)
file.close()

In [28]:
print("Accuracy score (validation): {0:.3f}".format(ABC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, ABC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, ABC.predict(x_test)))

Accuracy score (validation): 0.729
Confusion Matrix for Raandom Forests:
[[7674  316  392  650   21]
 [ 504 7932  161  721   50]
 [1836 1600 7772 1594   63]
 [2188 1082  702 8747   57]
 [   6    1   14    3   35]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.63      0.85      0.72      9053
           1       0.73      0.85      0.78      9368
           2       0.86      0.60      0.71     12865
           3       0.75      0.68      0.71     12776
           4       0.15      0.59      0.25        59

    accuracy                           0.73     44121
   macro avg       0.62      0.72      0.63     44121
weighted avg       0.75      0.73      0.73     44121



# SVM.svc(train很久不要用)

In [24]:
print("Accuracy score (validation): {0:.3f}".format(SVC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, SVC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, SVC.predict(x_test)))

Accuracy score (validation): 0.987
Confusion Matrix for Raandom Forests:
[[ 8951    14    44    98     0]
 [    4  9209    35    91     0]
 [   70    36 12749     3     0]
 [   65    77     5 12612     1]
 [    9    21     6    13     8]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      9107
           1       0.98      0.99      0.99      9339
           2       0.99      0.99      0.99     12858
           3       0.98      0.99      0.99     12760
           4       0.89      0.14      0.24        57

    accuracy                           0.99     44121
   macro avg       0.97      0.82      0.84     44121
weighted avg       0.99      0.99      0.99     44121

